In [37]:
import pandas as pd
import requests
import json
import time
from tqdm import tqdm

def getGameIds():
    headers = {
        "x-api-key" : "0TvQnueqKa5mxJntVWt0w4LpLfEkrV1Ta8rQBb9Z"
    }
    resultlist = []
    previous_data = None
    tournaments = pd.read_excel("data/target_tournament.xlsx", header=1)["id"]
    for tournament in tqdm(tournaments):
        apiResult = requests.get(f"https://prod-relapi.ewp.gg/persisted/gw/getCompletedEvents?hl=en-US&tournamentId={tournament}", headers=headers)
        if apiResult.status_code == 200:
            json = apiResult.json()
            if 'data' in json and len(json['data']) > 0:
                events = json["data"]['schedule']['events']
                for event in events:
                    match = event["match"]
                    games = event["games"]
                    blueteam = match["teams"][0]
                    redteam = match["teams"][1]
                    for idx, game in enumerate(games):
                        if "vods" in game and len(game["vods"]) > 0:
                            game_data = {
                                "startTime" : event["startTime"][:19]+"Z",
                                "leagueName": event["league"]["name"],
                                "matchId": match["id"],
                                "bestof": match["strategy"]["count"],
                                "blueteam_name" : blueteam["name"],
                                "blueteam_win" : blueteam["result"]["gameWins"],
                                "redteam_name" : redteam["name"],
                                "redteam_win" : redteam["result"]["gameWins"],
                                "gameNumberInAMatch": idx + 1, # 0이아니라 1부터 쓰려고
                                "gameId": game["id"]
                            }
                            game_data["gameNumberInAMatch"] = idx + 1 # 0이아니라 1부터 쓰려고
                            game_data["gameId"] = game["id"]
                            resultlist.append(game_data)
        time.sleep(0.1)
    game_ids_df = pd.DataFrame(resultlist).sort_values(by=["matchId", "gameNumberInAMatch"])
    return game_ids_df

In [38]:
game_ids = getGameIds() # 게임아이디가 들어간 리스트
game_ids

100%|██████████| 189/189 [02:50<00:00,  1.11it/s]


,startTime,leagueName,matchId,bestof,blueteam_name,blueteam_win,redteam_name,redteam_win,gameNumberInAMatch,gameId
920,2021-06-05T23:00:00Z,LCS,105658534675876639,1,Cloud9,0,100 Thieves,1,1,105658534675876640
947,2021-06-19T02:00:00Z,LCS,105658534675876641,1,100 Thieves,0,CLG,1,1,105658534675876642
923,2021-06-06T20:00:00Z,LCS,105658534675876643,1,Dignitas,0,100 Thieves,1,1,105658534675876644
942,2021-06-14T00:00:00Z,LCS,105658534675876645,1,Evil Geniuses LG,0,100 Thieves,1,1,105658534675876646
951,2021-06-20T00:00:00Z,LCS,105658534675876647,1,FlyQuest,0,100 Thieves,1,1,105658534675876648
...,...,...,...,...,...,...,...,...,...,...
73,2024-05-18T09:00:00Z,MSI,112274705999884340,5,Bilibili Gaming,3,T1,2,5,112274705999884345
74,2024-05-19T09:00:00Z,MSI,112274705999884346,5,Gen.G,3,Bilibili Gaming,1,1,112274705999884347
75,2024-05-19T09:00:00Z,MSI,112274705999884346,5,Gen.G,3,Bilibili Gaming,1,2,112274705999884348
76,2024-05-19T09:00:00Z,MSI,112274705999884346,5,Gen.G,3,Bilibili Gaming,1,3,112274705999884349


In [40]:
game_ids.to_excel("data/game_ids.xlsx", index=None)
game_ids

,startTime,leagueName,matchId,bestof,blueteam_name,blueteam_win,redteam_name,redteam_win,gameNumberInAMatch,gameId
920,2021-06-05T23:00:00Z,LCS,105658534675876639,1,Cloud9,0,100 Thieves,1,1,105658534675876640
947,2021-06-19T02:00:00Z,LCS,105658534675876641,1,100 Thieves,0,CLG,1,1,105658534675876642
923,2021-06-06T20:00:00Z,LCS,105658534675876643,1,Dignitas,0,100 Thieves,1,1,105658534675876644
942,2021-06-14T00:00:00Z,LCS,105658534675876645,1,Evil Geniuses LG,0,100 Thieves,1,1,105658534675876646
951,2021-06-20T00:00:00Z,LCS,105658534675876647,1,FlyQuest,0,100 Thieves,1,1,105658534675876648
...,...,...,...,...,...,...,...,...,...,...
73,2024-05-18T09:00:00Z,MSI,112274705999884340,5,Bilibili Gaming,3,T1,2,5,112274705999884345
74,2024-05-19T09:00:00Z,MSI,112274705999884346,5,Gen.G,3,Bilibili Gaming,1,1,112274705999884347
75,2024-05-19T09:00:00Z,MSI,112274705999884346,5,Gen.G,3,Bilibili Gaming,1,2,112274705999884348
76,2024-05-19T09:00:00Z,MSI,112274705999884346,5,Gen.G,3,Bilibili Gaming,1,3,112274705999884349
